In [1]:
import pandas as pd
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64

In [2]:
# Configuración de credenciales de Spotify
SPOTIFY_CLIENT_ID = "dfea28cb19364f0ca5c467a26be6f36d"
SPOTIFY_CLIENT_SECRET = "7eadcbfdd69c47158d87fe1b17ca53b1"

def get_spotify_token():
    url = "https://accounts.spotify.com/api/token"
    auth_string = f"{SPOTIFY_CLIENT_ID}:{SPOTIFY_CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

    headers = {
        "Authorization": f"Basic {auth_base64}",
        "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    token = response.json().get("access_token")

    if not token:
        print("⚠️ Error al obtener el token de Spotify:", response.json())

    return token

In [3]:
def get_album_cover(track_name, artist):
    # Función para obtener la portada del álbum de una canción usando la API de Spotify.
    token = get_spotify_token()
    if not token:
        return None

    url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {token}"}
    params = {"q": f"track:{track_name} artist:{artist}", "type": "track", "limit": 1}

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Error al obtener datos de Spotify: {response.status_code} - {response.text}")
        return None

    data = response.json()

    if "tracks" in data and "items" in data["tracks"] and len(data["tracks"]["items"]) > 0:
        return data["tracks"]["items"][0]["album"]["images"][0]["url"], data["tracks"]["items"][0]["external_urls"]["spotify"]
    
    print(f"No se encontró portada para: {track_name} - {artist}")
    return None, None

In [4]:
# Cargar el dataset de Spotify, limpiar valores nulos y ordenar por popularidad.
df = pd.read_csv("spotify_114k_tracks.csv")
df.dropna(inplace=True)
df = df.sort_values(by="popularity", ascending=False)
df = df.groupby(["track_name", "artists"], as_index=False).first()

if 'duration_ms' in df.columns:
    df["duration_s"] = df["duration_ms"] / 1000
    df.drop(columns=["duration_ms"], inplace=True)

In [5]:
# Crear una métrica combinada de características musicales
df["music_intensity"] = (
    (df["energy"] * 0.4) + 
    (df["danceability"] * 0.3) + 
    (df["valence"] * 0.2) + 
    (df["acousticness"] * -0.1) + 
    (df["instrumentalness"] * -0.2) + 
    (df["speechiness"] * 0.1)
)


In [6]:
# Normalización de características relevantes
features = ["danceability", "energy", "loudness", "speechiness", 
            "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

df[features] = df[features].fillna(df[features].mean())                                                                 
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[features] = scaler.fit_transform(df[features])

df["full_name"] = df["artists"] + " - " + df["track_name"]
df_scaled["full_name"] = df["full_name"]

In [7]:
# Crear widgets interactivos
search_box = widgets.Text(placeholder="Escribe artista o canción...")
results_box = widgets.Output()
recommendations_box = widgets.Output()

In [8]:
def recomendar_cancion(cancion_seleccionada):
    # Función para recomendar canciones basadas en similitudes de características musicales

    global df_scaled
    
    cancion_seleccionada_original = cancion_seleccionada.strip()
    cancion_seleccionada = cancion_seleccionada_original.lower()
    
    cancion = df_scaled[df_scaled["full_name"].str.lower().str.contains(cancion_seleccionada, regex=False)]
    if cancion.empty:
        with recommendations_box:
            clear_output(wait=True)
            print("Canción no encontrada en la base de datos.")
        return
    
    cancion = cancion.sort_values(by="popularity", ascending=False).head(1)
    cancion_features = cancion[features].values.reshape(1, -1)
    
    df_filtrado = df_scaled[df_scaled["full_name"] != cancion["full_name"].values[0]].copy()
    
    similitudes = cosine_similarity(cancion_features, df_filtrado[features].values)
    df_filtrado["similitud"] = similitudes[0]
    
    recomendaciones = df_filtrado[df_filtrado["similitud"] < 1.0]
    recomendaciones = recomendaciones.sort_values(by="similitud", ascending=False).head(5)
    
    with recommendations_box:
        clear_output(wait=True)
        print(f"\nRecomendaciones para: {cancion_seleccionada_original}\n")
        
        items = []
        for _, row in recomendaciones.iterrows():
            cover_url, spotify_url = get_album_cover(row["track_name"], row["artists"])
            similitud = round(row["similitud"] * 100, 2)
            
            track_name_cleaned = row['track_name'].replace("'", "").replace('"', '')
            song_name = widgets.HTML(f"<div style='text-align: center; padding-top: 10px; font-weight: bold;'>{track_name_cleaned}</div>")
            
            img = widgets.Image(value=requests.get(cover_url).content, format='png', width=180, height=180) if cover_url else widgets.Label("Imagen no encontrada")
            
            info = widgets.HTML(f"<div style='text-align: center;'>{row['artists']}<br>Similitud: {similitud}%</div>")
            
            img_link = widgets.HTML(f"<a href='{spotify_url}' target='_blank'><img src='{cover_url}' width='180' height='180'></a>")
            
            items.append(widgets.VBox([img_link, song_name, info], layout=widgets.Layout(margin='10px')))
        
        display(widgets.HBox(items, layout=widgets.Layout(justify_content='center')))

In [9]:
def actualizar_resultados(cambio):
    texto = cambio["new"].strip().lower()
    
    with results_box:
        clear_output(wait=True)
        if len(texto) > 1:
            palabras = texto.split()
            resultados = df[df["full_name"].apply(lambda x: all(palabra in x.lower() for palabra in palabras))]
            if not resultados.empty:
                botones = [widgets.Button(description=res, layout=widgets.Layout(width="100%")) 
                           for res in resultados["full_name"].head(10)]
                for btn in botones:
                    btn.on_click(lambda b, descripcion=btn.description: recomendar_cancion(descripcion))
                display(widgets.VBox(botones))
            else:
                print("No se encontraron resultados.")
        else:
            print("Por favor, ingresa más de una palabra para la búsqueda.")

In [10]:
search_box.observe(actualizar_resultados, names="value")

In [11]:
# Mostrar widgets
display(search_box, results_box, recommendations_box)

Text(value='', placeholder='Escribe artista o canción...')

Output()

Output()